In [3]:
import torch

Torch очень похож на numpy. По факту мы работаем с многомерными массивами, но torch ещё умеет пропускать градиент в отличии от numpy.
Преимущества torcha:
1) практически через что угодно что мы сделаем с массивами мы можем пропускать градиент (numpy так не умеет)
2) можем работать на видеокартах, а numpy может работать только на CPU

In [4]:
#создание тензоров
a = torch.tensor([1, 2, 3, 4])

In [5]:
a

tensor([1, 2, 3, 4])

In [6]:
#случайный тензор 3 x 4
a = torch.randn(3, 4)
b = torch.randn(3, 4)

In [7]:
a, b

(tensor([[-1.0004,  0.6067,  0.1148,  1.4075],
         [ 2.3550,  0.4260, -0.6478,  0.8963],
         [ 2.0639,  0.8225, -0.4270,  2.2974]]),
 tensor([[ 0.7700, -0.1788,  0.4128, -0.4855],
         [ 0.2197,  0.3653,  0.4531, -1.1720],
         [ 1.4967, -0.0416, -2.2063, -0.5028]]))

In [8]:
#операции с тензорами
a + b, a * b # * - поэлементное умножение

(tensor([[-0.2305,  0.4278,  0.5276,  0.9221],
         [ 2.5747,  0.7913, -0.1946, -0.2756],
         [ 3.5606,  0.7809, -2.6333,  1.7946]]),
 tensor([[-0.7703, -0.1085,  0.0474, -0.6833],
         [ 0.5174,  0.1556, -0.2935, -1.0505],
         [ 3.0891, -0.0342,  0.9422, -1.1551]]))

In [9]:
torch.sin( a / b)

tensor([[-0.9634,  0.2483,  0.2746, -0.2399],
        [-0.9617,  0.9192, -0.9900, -0.6924],
        [ 0.9816, -0.8015,  0.1924,  0.9898]])

In [10]:
# @ - матричное умножение
a = torch.randn(3, 4)
b = torch.randn(4, 5)
a @ b

tensor([[-2.5623, -3.0303, -1.5837, -1.3794,  0.5412],
        [ 4.0491, -1.7962,  0.4550, -2.4373, -1.5165],
        [-2.5446, -1.6552,  0.6774, -1.1274,  5.4905]])

In [11]:
#Все операции которые есть все по умолчанию поэлементные

In [12]:
torch.softmax(a, dim=1 ) #  dim - выбор в доль какой оси нам нужно суммирование в 1

tensor([[0.1938, 0.1969, 0.5064, 0.1029],
        [0.2223, 0.5889, 0.0490, 0.1398],
        [0.4640, 0.1612, 0.3016, 0.0731]])

In [13]:
#Сделаем сумму по первой координате и должны в итоге увидеть единицу
torch.softmax(a, dim=1 ).sum(dim=1)

tensor([1.0000, 1.0000, 1.0000])

In [14]:
torch.sigmoid(a)

tensor([[0.3086, 0.3121, 0.5385, 0.1916],
        [0.4919, 0.7195, 0.1760, 0.3784],
        [0.8399, 0.6458, 0.7733, 0.4526]])

In [15]:
#очень часто можно делать операции вызывая метод самого тензора 
a.sigmoid() #numpy так не умеет

tensor([[0.3086, 0.3121, 0.5385, 0.1916],
        [0.4919, 0.7195, 0.1760, 0.3784],
        [0.8399, 0.6458, 0.7733, 0.4526]])

Градиенты


In [16]:
'''Попробуем посчитать производную  чего - нибудь 
По умолчанию тензора не умеют считать градиенты
Нужно прописать спец условие reguires_grad=True
'''
a = torch.tensor([2.], requires_grad=True )

In [17]:
a

tensor([2.], requires_grad=True)

In [18]:
b = ( a **2 )
b
#Появляется такая штука grad_fn=<PowBackward0>

tensor([4.], grad_fn=<PowBackward0>)

In [19]:
#запустить ход назад
b.backward()


In [20]:
#у a появилось поле grad
a.grad

tensor([4.])

In [21]:
x = torch.ones((2, 3), requires_grad=True)
y = torch.randn(2, 3, 4, requires_grad=True)


In [22]:
l = torch.exp(y.sum(dim=-1) + x.tanh()).sum()

In [23]:
l.backward()

In [24]:
#градиенты такого же размера как и сами переменные
x.grad, y.grad
# по сути это dl/dx и dl/dy

(tensor([[ 0.1477,  0.4812, 17.1291],
         [12.0157,  0.1346,  0.2711]]),
 tensor([[[ 0.3516,  0.3516,  0.3516,  0.3516],
          [ 1.1457,  1.1457,  1.1457,  1.1457],
          [40.7860, 40.7860, 40.7860, 40.7860]],
 
         [[28.6106, 28.6106, 28.6106, 28.6106],
          [ 0.3204,  0.3204,  0.3204,  0.3204],
          [ 0.6456,  0.6456,  0.6456,  0.6456]]]))

In [ ]:
'''Если мы без переопределения x и y продифференцируем ещё раз, то
    там будут удвоенные значения этих (x.grad, y.grad) штук
    Т.е. когда у нас уже есть какой то градиент, который у нас скопился в тензоре
    и если мы делаем backward() ещё раз и к нам приходят другие градиенты, то
    они у нас суммируются с теми, которые уже записаны
    
    Для чего это сделано?
    Мы можем считать градиент от большого батча, несмотря на то, что в память он у нас не влезает.
    Мы просто сделаем последовательно несколько раз для тех порций этого батча, которые влезут в память,
    и потом получим суммированный градиент по ним записанный вот так.
    '''

*Как работать с видеокартой*

In [ ]:
'''
    Заходим в терминал. Команда для проверки, есть ли у нас видеокарты :
        nvidia-smi
( CUDA - это оболочка видеокарт, которая позволяет делать вычисления)
Если вы где - то запускаете процессы с видеокартами и у вас будут проблемы с тем что почему то у вас их не видит процесс, то

    В torch можно вызвать вот такую штуку:
    ( в ноутбуке ) torch.cuda.is_available()
    И если вдруг у вас выводится False, а не True хотя видеокарты у вас есть и вы поставили правильную версию torcha, 
    которая работает с видеокартами. Убедитесь, что вы поставили в терминале вот такую переменную:
    CUDA_VISIBLE_DEVICES когда запускали ноутбук
    
    Зададим её заново, если False
    export CUDA_VISIBLE_DEVICES=0 ( Если надо указать больше одной, то это делается через запятую CUDA_VISIBLE_DEVICES=0,2)
    import torch
    torch.cuda.is_available()
    Должно выводится True
    
    Чтобы посмотреть сколько устройств подключено : 
    import torch
    torch.cuda.device_count()
    
    
'''

In [26]:
# Этой строчкой мы обеспечиваем то, что ноутбук или код который вы делаете
# будет работать и на устройствах с cuda  и без неё ( на ноуте без видеокарты)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

In [27]:
# Когда мы завели device мы можем любой тензор, который сформировали, передать на device
x = torch.randn(100).to(device)
x
# В CPU сформировался тензор x и потом он передался на GPU

tensor([-0.0130, -0.8866,  0.1617,  0.4486, -1.2352, -0.4186,  1.0706,  0.5256,
        -0.4088,  0.1154,  1.9048, -1.2696, -0.6438,  0.1282, -0.7682, -0.4834,
        -0.5169,  1.1448,  0.0927, -0.3067, -1.3168, -1.4590,  0.6077,  0.1536,
         0.4071, -0.6900, -0.6623,  0.9732,  0.7853, -1.8664, -0.9130,  1.6030,
        -0.0393,  0.1202, -0.2639,  1.1850,  1.5046,  0.0207,  0.4625, -0.7809,
        -0.2143, -0.4364, -0.9191,  0.5898, -0.1005,  0.1370, -0.2385,  0.7873,
         0.4112, -0.9373, -1.9929,  0.0322,  1.3960, -0.9496,  0.4858,  2.3482,
         0.8455, -0.2917, -0.1063,  1.2779,  0.1413,  0.4522,  0.2463,  1.3411,
         1.7150, -1.4077,  0.0794, -1.8186,  0.2158,  0.8137,  0.7754,  0.6847,
         1.8134, -0.3728,  0.6606,  0.3880, -0.9869, -1.7057, -0.6317, -1.1884,
         0.7964, -0.6870, -1.0297,  0.9454,  1.9795,  0.2553,  0.6553, -1.2612,
        -0.4421,  0.8839,  2.3395, -1.5274,  0.9159,  1.5516,  0.5804,  0.5136,
         0.8261, -0.1642, -0.0059, -1.13

In [28]:
#Можно сделать ещё более правильно, а именно запросить сформировать этот тензор  на cuda
#То есть он сразу на GPU
x = torch.randn(10, 10, device=device)


In [29]:
# Вернуть что то обратно
x = x.cpu()

In [40]:
# !!! Как делать НЕ НАДО ( если вы работаете с видеокартами )
# Мы не знаем на какую карту прилетел это тензор
# И если вы так напишете, а у вас в доступе нет cuda, то все падает
x = torch.randn(10,10).cuda()
x

*Философия того, как вы должны работать с CUDA*

* вся предобработка данных происходит на CPU

* передаем сформированные бачи, которые уже готовы чтобы скормить их нейронке на CUDA

* вся нейронка живет на CUDA, все вычисления на CUDA

________

*Когда мы что то хотим обучать, все сложные трудоемкие вычисления делаются на видеокарте.*

*Все что касается подготовки данных происходит на CPU.*




In [ ]:
# Давайте возьмем большой массив, посчитаем сумму всех элементов в тензоре разными способами
#a = torch.randn(10000, 10000)

In [ ]:
#Не запускать, а то комп зависнет
# на CPU
# начнем с python
a_python = a.tolist()
%%time
sum(a_python[i][j] for i in range (10000) for j in range (10000))
#вывод: 9 сек

#тоже самое с torch
%%time
a.sum()
#вывод:+- что и numpy (ms)

#тоже самое с помощью numpy 
a_numpy = a.numpy()

%%time
a_numpy.sum()
#вывод: +- тоже что и torch (ms)

In [ ]:
#Тоже самое на GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

a = a.to(device)
a.sum()
#работает быстрее микросекунды

*Про размерности в тензоре*:

 (c, d, n, batch ) 
* c - канальная размерность ( для RGB - 3 )
* d, n - пространственные размерности 
* batch - размер бача

In [30]:
x = torch.randn(5, 5, requires_grad=True)
y = torch.randn(5, 5, requires_grad=False)
# Продифференцируем что - нибудь от них 


In [31]:
l = (x * y).sum()
l.backward()

In [33]:
x.grad, y.grad

(tensor([[ 0.6497, -0.1948, -0.4127, -1.0013,  0.3555],
         [ 1.3952, -0.3213,  2.7972, -0.5341, -1.6148],
         [-0.5331,  1.1943,  1.4406, -0.3406,  0.8890],
         [ 0.8703,  1.4760,  0.7143,  1.3597,  0.4103],
         [-2.2257,  1.1956, -3.4907, -0.7050,  0.5310]]),
 None)

*Контекстные менеджеры*
* torch.no_grad() - когда вы оборачиваете ваш кусок кода в этот контекстный менеджер, он запрещает любые подсчеты градиентов
     (т.е. нам нужно прогнать какие - то объекты через нейронку ,сделать для них предсказания, но при этом не считать градиенты)
* torch.inference_mode() - ещё более жесткая оптимизация, Скоторая ускоряет вычисления 

In [ ]:
#С no_grad() граф вычислений нем тянется ,поэтому мы не можем продифференцировать
with torch.no_grad():
    l = (x * y).sum()
    l.backward()

In [35]:
l
# Нет вот это штуки grad_fn=<PowBackward0>, которая говорит о том, что тащится градиент

tensor(-2.5996)

In [37]:
#конструкция с with - это контекстный менеджер, который как - то влияет на то, как выполняется код внутри него
with torch.inference_mode():
    l = (x * y).sum()
    

In [38]:
l

tensor(-2.5996)

*Лирическое отступление* 

Когда вы ставите какой  - то код (левый ) , ВНИМАТЕЛЬНО следите за ВЕРСИЯМИ

        ( иногда ошибки в коде лечатся обновленной версией torch  )

*Обучение нейронок*


In [74]:
#У torcha есть библиотеки 
from torchvision.datasets import MNIST
import torchvision.transforms as T

In [75]:
#Трейн и тест + конвертирует картинку в тензор с помощью transform=T.ToTensor()

In [87]:
train_set  = MNIST('.MNIST', transform=T.ToTensor(), train=True, download=True)
test_set  = MNIST('.MNIST', transform=T.ToTensor(), train=False, download=True)

* Dataset - сущность, которая умеет выдавать по индексу некоторый элемент
* DataLoader - утилита, которая нужна для того, чтобы формировать батчи 

In [88]:

from torch.utils.data import Dataset, DataLoader

In [89]:
len(train_set) #длина обучающей выборки


60000

In [91]:
#вызовем любой индекс 
out = train_set[1555]

In [92]:
out

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

(<PIL.Image.Image image mode=L size=28x28>, 9)
* <PIL.Image.Image image mode=L size=28x28> - сама картинка
* 9 - это класс, которому принадлежит картинка

In [93]:
#классы
train_set.classes

['0 - zero',
 '1 - one',
 '2 - two',
 '3 - three',
 '4 - four',
 '5 - five',
 '6 - six',
 '7 - seven',
 '8 - eight',
 '9 - nine']

In [94]:
image, label = out
image.size

<function Tensor.size>

In [97]:
image.shape

torch.Size([1, 28, 28])

*Давайте возьмем полносвязную сетку и научим её на MNIST*
____

Всё что касается нейронок лежит тут

In [51]:
from torch  import nn

In [52]:
nn.Module #  у всех нейронок есть этот базовый класс

torch.nn.modules.module.Module

### Напишем свой класс для нейронки

Следите за размерностями! Чтобы всё работало:

Вот вы подаете вектор на вход нужно чтобы матрица на которую вы будете умножать была соотвествующего размера,
иначем просто умножение на матрицу не сработает. В этой модели мы подам векторы, которые имеют размерность 
in_features

In [61]:
from torch import nn

class MLP(nn.Module):
    #in_features - число входных признаков, num_classes - число классов,
    #hidden_size - скрытая размерность, которая будет отвечать тому какая у нас размерность скрытых слоев
    #для промежуточных линейных слоев
    def __init__(self, in_features, num_classes, hidden_size):
        #поскольку мы наследуемся, то нужно вызвать конструктор у базового класса
        super().__init__()
        #тут определим саму нейронку
        #nn.Sequential - контейнер - последовательное применение слоев, которые вы ему подаете
        self.model = nn.Sequential(
            
            nn.Linear(in_features=in_features, out_features=hidden_size), 
            # линейный слой nn.Linear(in_features - входное число у слоя,out_features=hidden_size - число выходных -
            #это параметр hidden_size - который регулирует ширину нейронки)
            
            #функция активации
            nn.ReLU(),
            
            #ещё один линейный слой  ( по приколу скажем, что bias = False)
            nn.Linear(in_features=hidden_size, out_features=hidden_size, bias=False),
            
            #ещё одна функция активации
            nn.LeakyReLU(0.1),
            
            #последний линейный слой
            nn.Linear(in_features=hidden_size, out_features=num_classes)
            #nn.LogSoftmax()
            
        )
    
    # forward() - функция, которая определяет проход вперед по нейронке
    def forward(self, x):
        return self.model(x)

In [68]:
#скрафтим нейронку :)
IMG_SIZE = 28 # т.к.размер картинки 28 х 28 ( картинки мы будем вытягивать в длинный вектор )
NUM_CLASSES = 10 # кол- во классов ( можно было бы написать len(train_set.classes))
HIDDEN_SIZE = 64
model = MLP(in_features=IMG_SIZE ** 2, num_classes=NUM_CLASSES, hidden_size=HIDDEN_SIZE).to(device)
#Не забудьте закинуть модельку на device

In [69]:
#Наша моделька выглядит как то так
model

MLP(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=False)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [64]:
#посмотрим на линейный слой подробнее
model.model[0]

Linear(in_features=784, out_features=64, bias=True)

In [65]:
#Как выглядят параметры внутри линейного слоя
#Матрица весов
model.model[0].weight, model.model[0].weight.shape

(Parameter containing:
 tensor([[-0.0193,  0.0347,  0.0179,  ..., -0.0147,  0.0234, -0.0155],
         [-0.0054, -0.0325,  0.0201,  ..., -0.0214, -0.0023,  0.0146],
         [-0.0120,  0.0109, -0.0233,  ...,  0.0133,  0.0321, -0.0134],
         ...,
         [ 0.0303,  0.0349,  0.0338,  ...,  0.0315, -0.0321, -0.0315],
         [ 0.0090,  0.0150, -0.0285,  ..., -0.0290,  0.0242,  0.0136],
         [-0.0241, -0.0283,  0.0140,  ...,  0.0191, -0.0082,  0.0257]],
        requires_grad=True),
 torch.Size([64, 784]))

In [66]:
#bias (добавка) - это b из ( wX + b)
model.model[0].bias, model.model[0].bias.shape
#Мы к каждой из 64 коорд выхода добавляем свой bias ( он тоже обучаемый )

(Parameter containing:
 tensor([-0.0119,  0.0059, -0.0307, -0.0207, -0.0295, -0.0249, -0.0328, -0.0165,
          0.0318,  0.0327, -0.0298,  0.0029,  0.0165, -0.0014, -0.0294, -0.0137,
          0.0330,  0.0330, -0.0326,  0.0187,  0.0214,  0.0018, -0.0150, -0.0355,
         -0.0025,  0.0022,  0.0034,  0.0291,  0.0163, -0.0142,  0.0019, -0.0041,
          0.0048,  0.0260,  0.0112, -0.0276,  0.0112, -0.0262,  0.0351,  0.0163,
         -0.0183,  0.0128, -0.0135,  0.0132, -0.0198,  0.0063,  0.0065, -0.0321,
          0.0067, -0.0299,  0.0286, -0.0147,  0.0189,  0.0092,  0.0058, -0.0163,
         -0.0330,  0.0270, -0.0082, -0.0170, -0.0156, -0.0184, -0.0307,  0.0269],
        requires_grad=True),
 torch.Size([64]))

In [67]:
#Если попробуем обратится к слою, где bias=False - то либо упадет, либо None
model.model[2].bias


*Резюме к этому моменту*

*Мы просто инициализировали модель, т.е. там лежат какие то случайные веса которые как - то инициализированы*

_____
*Нам нужно ещё штуку сделать с данными*

DataLoader - утилита, которая нужна для того, чтобы формировать батчи ( т.е. мы будем учить нашу нейронку с помощью mini-batch и градиентного спуска и можно автоматически формировать батчи )

Эта штука берет наш датасет, перемешивает там объекты ( т.к. нам надо брать каждый раз случайные объекты, а то нейронка
будет себя не оч чувствовать). Она сама будет являтся итератором, мы сможем по ней пройтись и нам будут выдаваться батчи из неё.

У нас будет отдельный DataLoader для обучающей выборки и отдельный DataLoader для тестовой

Есть параметр num_workers, который определяет во сколько потоков вы хотите обрабатывать ваши данные.
Условно говоря это происходит так:
        У вас идет обучение - работает CUDA, которая гоняет тензора по вашей нейронке  и 
        параллельно в этот момент у вас есть DataLoader, который готовит ей следующие батчи
        за счет рабочих, которые у него есть ( он вызывает у датасета по индексу элементы и 
        пока нейронка обрабатывает старые батчи, он делает новые ) 

batch_size - важный гиперпараметр в обучении
shuffle - мешает обучающую выборку
drop_last - убирает последний батч, который оказался неполным

In [98]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
#В тесте нет смысла мешать выборку, поэтому shuffle=False
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [100]:
#Демонстрация работы DataLoader
for images, labels in train_loader:
    print(images.shape, labels.shape)
    break
    #images имеют размер (64 - кол-во объектов в батче,1 - канал, 28 * 28 )

torch.Size([64, 1, 28, 28]) torch.Size([64])


*Вернемся к нашей модельке*

In [104]:
IMG_SIZE = 28 
NUM_CLASSES = 10 
HIDDEN_SIZE = 64
NUM_EPOCHS = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = MLP(in_features=IMG_SIZE ** 2, num_classes=NUM_CLASSES, hidden_size=HIDDEN_SIZE).to(device)
#Полезная штука при оптимизации ( lr - шаг градиентного спуска)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #model.parameters() - генератор, который выплевывает параметры модели
#Функция потерь ( nn.CrossEntropyLoss() - которому мы будем выдавать логиты )
criterion = nn.CrossEntropyLoss() # nn.NLLLoss()

Супер кратко как будет происходить обучение

У нас есть внешний цикл из эпох

Эпоха - промежуток времени за который мы проходим целый датасет.

    Как проходит само обучение :
    Мы берем датасет, перемешиваем его. Дальше вот он у нас перемешенный и мы из него выплевываем батчи, и 
    когда мы после одного перемешивания прошлись по всем батчам - то вот прошла одна эпоха. Т.е. в одной эпохе
    какое то количество шагов градиентного спуска, которые её характеризуют.

In [106]:
for epoch in range(1, NUM_EPOCHS + 1):
    #делаем эпохи по обучающей выборке 
    for images, labels in train_loader:
        #сначала закинем тензора на CUDA
        images = images.to(device)
        labels = labels.to(device)
        #Зануляет градиенты которые были в модели на данный момент. Если мы их не занулим, то новые градиенты 
        #просуммируются со старыми и мы получим немного не то направление в котором хотели двигаться
        optimizer.zero_grad()
        #вызываем модель по хорошему нужно так model.forward(), но обычно все пишут так, передаем ейм картинки, а на 
        #выходе получаем логиты 
        logits = model(torch.flatten(images, start_dim=1)) #схлопываем картинки тут с помощью torch.flatten(images, start_dim=1)
        #где start_dim=1 - мы сохраняем размерность батча, который идет первым, а остальное схлопнуть в одно
        loss = criterion(logits, labels)
        loss.backward()
        #делаем шаг градиентного спуска
        optimizer.step()
        #печатаем loss
        print(loss.item()) # с помощью .item() скалярный тензор преобразовываем во float
        #Все грохнется, если не схлопнули картинки
        

0.06660879403352737
0.13612134754657745
0.12195853888988495
0.06423068046569824
0.08542574942111969
0.03132500872015953
0.0951831117272377
0.2667919099330902
0.18502750992774963
0.10635750740766525
0.05006919056177139
0.06963323801755905
0.06214345619082451
0.11402523517608643
0.18952755630016327
0.059479594230651855
0.09856808185577393
0.13854095339775085
0.14596118032932281
0.08666568249464035
0.2276192605495453
0.05972415208816528
0.08181630074977875
0.1452317237854004
0.3045678734779358
0.09502623975276947
0.108859121799469
0.04306537285447121
0.1713477224111557
0.10613256692886353
0.08020009100437164
0.10018914937973022
0.09567076712846756
0.1304338574409485
0.15464608371257782
0.1291143149137497
0.1749298870563507
0.047559890896081924
0.05669552460312843
0.06227383762598038
0.18299855291843414
0.13301099836826324
0.02748379483819008
0.16030536592006683
0.06015905365347862
0.2790398597717285
0.057607702910900116
0.14542920887470245
0.19764065742492676
0.13584725558757782
0.2326414